# Домашнее задание 2

В первую очередь запускаю проверку по PEP-8:

In [1]:
%load_ext pycodestyle_magic
%pycodestyle_on

Потом в отдельную ячейку выношу все импорты и инстоллы, чтобы они не мешали дальше:

In [2]:
# ! pip3 install pycodestyle flake8 pycodestyle_magic
# ! pip install pymystem3
# ! pip install pymorphy2
# ! pip install nltk
# nltk.download('punkt')
import json
import nltk
import re
from collections import Counter
from pymystem3 import Mystem
from pprint import pprint
from nltk.tokenize import word_tokenize
from nltk import bigrams
from nltk.util import ngrams
from pymorphy2 import MorphAnalyzer

**Первое задание**

* распарсить с помощью mystem
* замерить время работы
* сохранить результат в json

Здесь беру только отрывок - один абзац, иначе слишком долго.

In [3]:
m = Mystem()
with open('Отрывок.txt', encoding='UTF-8') as f:
    text = f.read()
print('Время выполнения парсинга:')
%time  analysis = m.analyze(text)
with open('Mystem_analysis.json', 'w', encoding='utf-8') as f:
    json.dump(analysis, f, ensure_ascii=False)
print('\n' + 'Готово, данные анализа записаны в файл "Mystem_analysis.json"')

Время выполнения парсинга:
Wall time: 4.8 s

Готово, данные анализа записаны в файл "Mystem_analysis.json"


**Второе задание**

Измерение времени работы пришлось заккоментировать, т.к. с ним почему-то не работают выводы в следующих ячейках.

Сначала токенизиую текст уже всей книги через nltk. Под решёткой оставляю вариант с токенизацией всего подряд, но мне кажется, нет смысла дальше анализировать часть речи у знаков препинания. Для анализа беру только первое значение в случае, если их несколько. Извлекаю из тега лемму и часть речи. Записываю в файл json.

In [4]:
# %%timeit
morph = MorphAnalyzer()
with open('По ту сторону поводка.txt', encoding='UTF-8') as f:
    text = f.read()
# words = word_tokenize(text)  так остаётся пунктуация
words = [w.lower() for w in word_tokenize(text) if w.isalpha()]  # а так нет
analysis_2 = []
pair = {}
for word in words:
    result = morph.parse(word)
    first = result[0]
    lemma = first.normal_form
    pos = first.tag.POS
    pair = dict([(lemma, pos)])
    analysis_2.append(pair)
with open('Pymorphy_analysis.json', 'w', encoding='utf-8') as f:
    json.dump(analysis_2, f, ensure_ascii=False)
print('\n' + 'Готово, данные анализа записаны в файл "Pymorphy_analysis.json"')
# print('\n' + 'Время выполнения задач:')


Готово, данные анализа записаны в файл "Pymorphy_analysis.json"


**Третье задание, часть 1**

Не самый красивый код, но мне захотелось сделать красивый русскоязычный вывод, а не просто сокращенную части речи и число.

NB: слова с пометой INFN здесь считаются за глаголы.

NB2: числительные почему-то парсятся как NOUN

In [5]:
words_nums = re.findall(r"[\w]+", text)
verb = []
adj = []
noun = []
pron = []
conj = []
prep = []
inf = []
adv = []
for i in analysis_2:
    for key in i.keys():
        if i[key] == 'VERB':
            verb.append(i[key])
        if i[key] == 'ADJF':
            adj.append(i[key])
        if i[key] == 'NOUN':
            noun.append(i[key])
        if i[key] == 'NPRO':
            pron.append(i[key])
        if i[key] == 'CONJ':
            conj.append(i[key])
        if i[key] == 'PREP':
            prep.append(i[key])
        if i[key] == 'INFN':
            inf.append(i[key])
        if i[key] == 'ADVB':
            adv.append(i[key])
print('Доля глаголов в тексте: ', (len(verb) + len(inf)) / len(words_nums))
print('Доля прилагательных в тексте: ', len(adj) / len(words_nums))
print('Доля существительных в тексте: ', len(noun) / len(words_nums))
print('Доля наречий в тексте: ', len(adv) / len(words_nums))
print('Доля местоимений в тексте: ', len(pron) / len(words_nums))
print('Доля союзов в тексте: ', len(conj) / len(words_nums))
print('Доля предлогов в тексте: ', len(prep) / len(words_nums))

Доля глаголов в тексте:  0.13562924735578152
Доля прилагательных в тексте:  0.1295488222369833
Доля существительных в тексте:  0.2617239793572122
Доля наречий в тексте:  0.04752950794542946
Доля местоимений в тексте:  0.06880588626028307
Доля союзов в тексте:  0.10641254917990906
Доля предлогов в тексте:  0.10826222472024935


**Третье задание, часть 2**

Раскидываю слова по спискам глаголов и наречий по результатам парсинга, затем каунтером нахожу самые частотные (использую леммы, а не формы слов)

In [6]:
verbs_2 = []
advs_2 = []
for word in words:
    result = morph.parse(word)
    first = result[0]
    if first.tag.POS == 'VERB':
        verbs_2.append(first.normal_form)
    if first.tag.POS == 'ADVB':
        advs_2.append(first.normal_form)
top_verbs = Counter(verbs_2).most_common(20)
top_advs = Counter(advs_2).most_common(20)
print('Топ-20 глаголов:', '\n')
for v in top_verbs:
    print(str(v).strip(')('))
print('\n', 'Топ-20 наречий:', '\n')
for a in top_advs:
    print(str(a).strip(')('))

Топ-20 глаголов: 

'мочь', 675
'быть', 636
'хотеть', 168
'делать', 138
'знать', 124
'иметь', 117
'любить', 90
'стать', 88
'использовать', 84
'играть', 81
'видеть', 75
'сказать', 74
'говорить', 71
'пытаться', 69
'думать', 62
'сделать', 61
'означать', 59
'смочь', 56
'жить', 55
'являться', 54

 Топ-20 наречий: 

'более', 114
'поэтому', 113
'несколько', 99
'тогда', 95
'никогда', 95
'настолько', 82
'затем', 78
'всегда', 76
'хорошо', 75
'точно', 75
'пока', 74
'вперёд', 71
'всего', 70
'назад', 70
'насколько', 69
'часто', 69
'прямо', 66
'много', 62
'потому', 62
'очень', 61


Так как исследуемая книга - про поведение и воспитание собак, я из интереса добавила еще и топ существительных. "Собака" оказалась на первом месте с большим отрывом! А дальше идут "человек" и "друг". Очень мило

In [7]:
nouns_2 = []
for word in words:
    result = morph.parse(word)
    first = result[0]
    if first.tag.POS == 'NOUN':
        nouns_2.append(first.normal_form)
top_nouns = Counter(nouns_2).most_common(20)
print('Топ-20 существительных:', '\n')
for n in top_nouns:
    print(str(n).strip(')('))

Топ-20 существительных: 

'собака', 2255
'человек', 600
'друг', 239
'поведение', 226
'время', 226
'животное', 185
'щенок', 182
'вид', 179
'внимание', 152
'дом', 142
'рука', 142
'раз', 137
'игра', 136
'имя', 134
'мяч', 134
'люк', 130
'звук', 130
'ребёнок', 129
'место', 129
'книга', 127


**Четвертое задание**

Сначала делаю список всех лемм, вытащенных из Pymorphy.

У nltk есть отдельный способ получить биграммы, и отдельный способ для любых n-грамм. Я использовала оба, чтобы получить опыт с ними, хотя у ngrams можно задать любое число.

In [8]:
lemmas = []
for word in words:
    result = morph.parse(word)
    first = result[0]
    lemma = first.normal_form
    lemmas.append(lemma)
bi = list(bigrams(lemmas))
tri = ngrams(lemmas, 3)
top_bi = Counter(bi).most_common(25)
top_tri = Counter(tri).most_common(25)
print('Топ-25 биграмм:', '\n')
for b in top_bi:
    print(b)
print('\n', 'Топ-25 триграмм:', '\n')
for t in top_tri:
    print(t)

Топ-25 биграмм: 

(('тот', 'что'), 203)
(('собака', 'и'), 148)
(('ваш', 'собака'), 139)
(('тот', 'как'), 129)
(('то', 'что'), 127)
(('свой', 'собака'), 119)
(('в', 'тот'), 112)
(('не', 'мочь'), 104)
(('я', 'не'), 102)
(('собака', 'который'), 100)
(('с', 'собака'), 99)
(('человек', 'и'), 94)
(('тот', 'чтобы'), 94)
(('и', 'не'), 93)
(('один', 'из'), 86)
(('как', 'и'), 85)
(('мочь', 'быть'), 82)
(('собака', 'в'), 81)
(('о', 'тот'), 80)
(('если', 'вы'), 78)
(('другой', 'собака'), 78)
(('так', 'же'), 77)
(('и', 'в'), 76)
(('тот', 'же'), 76)
(('это', 'не'), 75)

 Топ-25 триграмм: 

(('в', 'конец', 'конец'), 55)
(('так', 'же', 'как'), 47)
(('на', 'самый', 'дело'), 45)
(('человек', 'и', 'собака'), 44)
(('точно', 'так', 'же'), 43)
(('о', 'тот', 'что'), 39)
(('по', 'отношение', 'к'), 38)
(('после', 'тот', 'как'), 37)
(('в', 'тот', 'что'), 35)
(('для', 'тот', 'чтобы'), 28)
(('друг', 'с', 'друг'), 28)
(('на', 'то', 'что'), 26)
(('до', 'тот', 'как'), 25)
(('с', 'свой', 'собака'), 25)
(('в', 'то', '

Результат: в биграммах кроме грамматических конструкций встречаются и частотные словосочетания, характерные как раз для книги про собак, а в триграммах уже почти только распростанённые устойчивые выражения (*в конце концов, так же как, на самом деле* и тд). Однако, на четвёртом месте идёт *человек и собака*, что кажется мне очень милым.